# 🎬 영화 리뷰 감정 분석 - TF-IDF & BoW 방식

## 📋 목차
1. [환경 설정 및 라이브러리 임포트](#1-환경-설정-및-라이브러리-임포트)
2. [데이터 다운로드 및 전처리](#2-데이터-다운로드-및-전처리)
3. [데이터셋 로딩 및 확인](#3-데이터셋-로딩-및-확인)
4. [텍스트 벡터화 (TF-IDF)](#4-텍스트-벡터화-tf-idf)
5. [모델 구성 및 훈련](#5-모델-구성-및-훈련)
6. [모델 평가 및 예측](#6-모델-평가-및-예측)

---

## 🎯 프로젝트 개요
- **목표**: IMDb 영화 리뷰 데이터를 사용한 긍정/부정 감정 분석
- **방법**: TF-IDF (Term Frequency-Inverse Document Frequency) 벡터화
- **특징**: N-gram (2-gram) 사용으로 단어 조합 패턴 학습
- **모델**: 간단한 Dense Neural Network


## 1. 환경 설정 및 라이브러리 임포트

필요한 라이브러리들을 임포트합니다:
- **requests**: 데이터 다운로드
- **tensorflow/keras**: 딥러닝 모델 구성
- **TextVectorization**: 텍스트 벡터화
- **기타**: 파일 처리, 시스템 관련


In [2]:
import requests
import subprocess
import re
import string
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization
import os, pathlib, shutil, random
import keras
from keras import layers, models
import numpy as np

print("TensorFlow 버전:", tf.__version__)
print("Keras 버전:", keras.__version__)
print("✅ 라이브러리 임포트 완료!")


TensorFlow 버전: 2.15.1
Keras 버전: 2.15.0
✅ 라이브러리 임포트 완료!


## 2. 데이터 다운로드 및 전처리

IMDb 영화 리뷰 데이터셋을 다운로드하고 전처리합니다:

### 📁 데이터셋 정보
- **출처**: Stanford AI Lab
- **크기**: 약 84MB (압축)
- **구성**: 25,000개 훈련 리뷰 + 25,000개 테스트 리뷰
- **라벨**: pos(긍정), neg(부정)

### 🔄 전처리 과정
1. **다운로드**: 압축 파일 다운로드
2. **압축 해제**: tar.gz 파일 해제
3. **라벨링**: train/validation 분할 (80:20)


In [3]:
# 데이터 다운로드 함수
def download():
    """IMDb 데이터셋을 다운로드합니다."""
    url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
    file_name = "aclImdb_v1.tar.gz"
    
    print("📥 데이터셋 다운로드 시작...")
    response = requests.get(url, stream=True)  # 스트리밍 방식으로 다운로드
    with open(file_name, "wb") as file:
        for chunk in response.iter_content(chunk_size=8192):  # 8KB씩 다운로드
            file.write(chunk)
    
    print("✅ 다운로드 완료!")

# 압축 해제 함수
def release():
    """다운로드한 압축 파일을 해제합니다."""
    print("📂 압축 해제 중...")
    subprocess.run(["tar", "-xvzf", "aclImdb_v1.tar.gz"], shell=True)
    print("✅ 압축 해제 완료!")

# 데이터 라벨링 및 분할 함수
def labeling():
    """train 데이터를 train/validation으로 분할합니다."""
    print("🏷️ 데이터 라벨링 및 분할 중...")
    base_dir = pathlib.Path("aclImdb")
    val_dir = base_dir / "val"
    train_dir = base_dir / "train"
    
    for category in ("neg", "pos"):
        os.makedirs(val_dir / category, exist_ok=True)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)  # 재현 가능한 랜덤 셔플
        num_val_samples = int(0.2 * len(files))  # 20%를 validation으로
        val_files = files[-num_val_samples:]
        
        for fname in val_files:
            shutil.move(train_dir / category / fname, val_dir / category / fname)
    
    print("✅ 라벨링 및 분할 완료!")

print("📋 데이터 전처리 함수들이 정의되었습니다.")
print("💡 필요시 다음 함수들을 호출하세요:")
print("   - download(): 데이터셋 다운로드")
print("   - release(): 압축 해제") 
print("   - labeling(): 데이터 분할")


📋 데이터 전처리 함수들이 정의되었습니다.
💡 필요시 다음 함수들을 호출하세요:
   - download(): 데이터셋 다운로드
   - release(): 압축 해제
   - labeling(): 데이터 분할


## 3. 데이터셋 로딩 및 확인

Keras의 `text_dataset_from_directory`를 사용하여 텍스트 데이터를 로딩합니다.

### 📊 데이터셋 구조
- **폴더 구조**: `aclImdb/train/pos`, `aclImdb/train/neg` 등
- **배치 크기**: 32개씩 처리
- **라벨**: 0(부정), 1(긍정) - 폴더명 알파벳 순서


In [4]:
# 데이터셋 설정
batch_size = 32  # 한 번에 처리할 데이터 개수

print("📂 데이터셋 로딩 중...")

# 훈련용 데이터셋
train_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/train",
    batch_size=batch_size
)

# 검증용 데이터셋  
val_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/val",
    batch_size=batch_size
)

# 테스트용 데이터셋
test_ds = keras.utils.text_dataset_from_directory(
    "../../data/aclImdb/test",
    batch_size=batch_size
)

print("✅ 데이터셋 로딩 완료!")
print(f"📊 배치 크기: {batch_size}")


📂 데이터셋 로딩 중...
Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
✅ 데이터셋 로딩 완료!
📊 배치 크기: 32


In [5]:
# 데이터 구조 확인
print("🔍 데이터 구조 확인:")
print("-" * 50)

for inputs, targets in train_ds:
    print("📝 입력 데이터 (리뷰 텍스트):")
    print(f"   - 형태: {inputs.shape}")
    print(f"   - 타입: {inputs.dtype}")
    print()
    
    print("🎯 타겟 데이터 (라벨):")
    print(f"   - 형태: {targets.shape}")
    print(f"   - 타입: {targets.dtype}")
    print()
    
    print("📖 샘플 데이터 (처음 3개):")
    for i in range(3):
        print(f"   리뷰 {i+1}: {inputs[i].numpy().decode('utf-8')[:100]}...")
        print(f"   라벨 {i+1}: {targets[i].numpy()} ({'긍정' if targets[i].numpy() == 1 else '부정'})")
        print()
    
    break  # 첫 번째 배치만 확인

print("💡 라벨 정보: 0=부정(neg), 1=긍정(pos)")


🔍 데이터 구조 확인:
--------------------------------------------------
📝 입력 데이터 (리뷰 텍스트):
   - 형태: (32,)
   - 타입: <dtype: 'string'>

🎯 타겟 데이터 (라벨):
   - 형태: (32,)
   - 타입: <dtype: 'int32'>

📖 샘플 데이터 (처음 3개):
   리뷰 1: This is a truly magnificent and heartwrenching film!!!! Ripstein's locations are spectacular, extrem...
   라벨 1: 1 (긍정)

   리뷰 2: Jimmy Cagney races by your eyes constantly in this story of a stage-producer who is vigorously strug...
   라벨 2: 1 (긍정)

   리뷰 3: SWING! is an important film because it's one of the remaining Black-produced and acted films from th...
   라벨 3: 0 (부정)

💡 라벨 정보: 0=부정(neg), 1=긍정(pos)


## 4. 텍스트 벡터화 (TF-IDF)

### 🔤 TF-IDF란?
- **TF (Term Frequency)**: 문서 내 단어 빈도
- **IDF (Inverse Document Frequency)**: 단어의 희귀성 
- **조합**: 자주 나오지만 특별한 의미를 가진 단어에 높은 가중치

### ⚙️ 설정 파라미터
- **max_tokens**: 20,000개 (가장 빈번한 단어들)
- **output_mode**: "tf_idf" (TF-IDF 가중치 적용)
- **ngrams**: 2 (단어 2개 조합도 고려)

### 🎯 N-gram의 효과
- **1-gram**: "good", "movie" 
- **2-gram**: "good movie", "very good"
- 문맥 정보를 더 잘 캡처할 수 있음


In [7]:
# TF-IDF 벡터화 설정
print("🔤 TF-IDF 벡터화 설정 중...")

text_vectorization = TextVectorization(
    max_tokens=20000,           # 자주 사용하는 단어 20,000개만 사용
    output_mode="tf_idf",       # TF-IDF 가중치 적용
    ngrams=2                    # 2-gram 사용 (단어 조합 고려)
)

print("✅ 벡터화 설정 완료!")
print("📊 설정 정보:")
print(f"   - 최대 토큰 수: {20000:,}")
print(f"   - 출력 모드: TF-IDF")
print(f"   - N-gram: 2 (단어 조합 고려)")


🔤 TF-IDF 벡터화 설정 중...
✅ 벡터화 설정 완료!
📊 설정 정보:
   - 최대 토큰 수: 20,000
   - 출력 모드: TF-IDF
   - N-gram: 2 (단어 조합 고려)


In [9]:
# 어휘사전 생성 (텍스트만 추출하여 학습)
print("📚 어휘사전 생성 중...")
text_only_train_ds = train_ds.map(lambda x, y: x)  # 텍스트만 추출
text_vectorization.adapt(text_only_train_ds)       # 어휘사전 생성
print("✅ 어휘사전 생성 완료!")

# 모든 데이터셋에 벡터화 적용
print("🔄 데이터셋 벡터화 중...")
print("   - 멀티프로세싱으로 빠른 처리...")

# 멀티프로세싱을 사용한 벡터화 (CPU 코어 4개 활용)
binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)
binary_1gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)
binary_1gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), 
    num_parallel_calls=4
)

print("✅ 모든 데이터셋 벡터화 완료!")


📚 어휘사전 생성 중...
✅ 어휘사전 생성 완료!
🔄 데이터셋 벡터화 중...
   - 멀티프로세싱으로 빠른 처리...
✅ 모든 데이터셋 벡터화 완료!


In [10]:
# 벡터화 결과 확인
print("🔍 벡터화 결과 확인:")
print("-" * 50)

for inputs, targets in binary_1gram_train_ds:
    print("📊 벡터화된 입력 데이터:")
    print(f"   - 형태: {inputs.shape}")
    print(f"   - 타입: {inputs.dtype}")
    print(f"   - 값 범위: {inputs.numpy().min():.6f} ~ {inputs.numpy().max():.6f}")
    print()
    
    print("🎯 타겟 데이터:")
    print(f"   - 형태: {targets.shape}")
    print(f"   - 타입: {targets.dtype}")
    print()
    
    print("📈 첫 번째 샘플의 TF-IDF 값 (0이 아닌 값들):")
    first_sample = inputs[0].numpy()
    non_zero_indices = np.where(first_sample > 0)[0]
    print(f"   - 활성화된 특성 수: {len(non_zero_indices)}/{len(first_sample)}")
    print(f"   - 처음 10개 비0 값: {first_sample[non_zero_indices[:10]]}")
    print()
    
    break

print("💡 TF-IDF 특징:")
print("   - 값이 클수록 해당 단어/구문이 문서에서 중요함")
print("   - 대부분의 값이 0 (희소 행렬)")
print("   - 문서별로 고유한 패턴을 가짐")


🔍 벡터화 결과 확인:
--------------------------------------------------
📊 벡터화된 입력 데이터:
   - 형태: (32, 20000)
   - 타입: <dtype: 'float32'>
   - 값 범위: 0.000000 ~ 3422.129639

🎯 타겟 데이터:
   - 형태: (32,)
   - 타입: <dtype: 'int32'>

📈 첫 번째 샘플의 TF-IDF 값 (0이 아닌 값들):
   - 활성화된 특성 수: 104/20000
   - 처음 10개 비0 값: [293.1731      6.974498    0.7111458   4.2621336   2.8794088   1.4509387
   3.7522266   0.7591958   4.6798143   1.6230419]

💡 TF-IDF 특징:
   - 값이 클수록 해당 단어/구문이 문서에서 중요함
   - 대부분의 값이 0 (희소 행렬)
   - 문서별로 고유한 패턴을 가짐


## 5. 모델 구성 및 훈련

### 🧠 모델 아키텍처
- **입력층**: TF-IDF 벡터 (20,000 차원)
- **은닉층**: Dense(16) + ReLU 활성화
- **정규화**: Dropout(0.5) - 과적합 방지
- **출력층**: Dense(1) + Sigmoid - 이진 분류

### 🎯 모델 특징
- **간단한 구조**: Dense 레이어만 사용
- **효율성**: TF-IDF로 이미 의미있는 특성 추출됨
- **정규화**: 드롭아웃으로 일반화 성능 향상


In [12]:
# 모델 생성 함수
def create_model(max_tokens=20000, hidden_dim=16):
    """
    TF-IDF 기반 감정 분석 모델을 생성합니다.
    
    Args:
        max_tokens (int): 입력 벡터의 차원수
        hidden_dim (int): 은닉층의 뉴런 수
    
    Returns:
        keras.Model: 컴파일된 모델
    """
    # 모델 구조 정의
    inputs = keras.Input(shape=(max_tokens,), name='tfidf_input')
    
    # 은닉층 (ReLU 활성화)
    x = layers.Dense(hidden_dim, activation='relu', name='hidden_layer')(inputs)
    
    # 드롭아웃 (과적합 방지)
    x = layers.Dropout(0.5, name='dropout')(x)
    
    # 출력층 (시그모이드 활성화 - 이진 분류)
    outputs = layers.Dense(1, activation='sigmoid', name='output_layer')(x)
    
    # 모델 생성
    model = keras.Model(inputs, outputs, name='tfidf_sentiment_model')
    
    # 컴파일
    model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# 모델 생성 및 요약
print("🧠 모델 생성 중...")
model = create_model()
print("✅ 모델 생성 완료!")
print()

print("📋 모델 구조:")
model.summary()


🧠 모델 생성 중...
✅ 모델 생성 완료!

📋 모델 구조:
Model: "tfidf_sentiment_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tfidf_input (InputLayer)    [(None, 20000)]           0         
                                                                 
 hidden_layer (Dense)        (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 output_layer (Dense)        (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# 콜백 설정 (최적 모델 저장)
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "binary_2gram_tfidf.keras",
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=3,
        verbose=1,
        restore_best_weights=True
    )
]

print("🚀 모델 훈련 시작!")
print("=" * 50)

# 모델 훈련
# cache(): 첫 번째 에포크에서 전처리를 한 번만 하고 메모리에 캐싱
history = model.fit(
    binary_1gram_train_ds.cache(),
    validation_data=binary_1gram_val_ds,
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

print("✅ 모델 훈련 완료!")
print(f"💾 최적 모델이 'binary_2gram_tfidf.keras'로 저장되었습니다.")


🚀 모델 훈련 시작!
Epoch 1/10
615/625 [============================>.] - ETA: 0s - loss: 0.3765 - accuracy: 0.8399
Epoch 1: val_accuracy improved from -inf to 0.88360, saving model to binary_2gram_tfidf.keras
625/625 [==============================] - 7s 11ms/step - loss: 0.3760 - accuracy: 0.8403 - val_loss: 0.3047 - val_accuracy: 0.8836
Epoch 2/10
621/625 [============================>.] - ETA: 0s - loss: 0.3130 - accuracy: 0.8725
Epoch 2: val_accuracy did not improve from 0.88360
625/625 [==============================] - 4s 6ms/step - loss: 0.3135 - accuracy: 0.8724 - val_loss: 0.3309 - val_accuracy: 0.8704
Epoch 3/10
621/625 [============================>.] - ETA: 0s - loss: 0.2842 - accuracy: 0.8854
Epoch 3: val_accuracy did not improve from 0.88360
625/625 [==============================] - 4s 6ms/step - loss: 0.2839 - accuracy: 0.8854 - val_loss: 0.3190 - val_accuracy: 0.8814
Epoch 4/10
616/625 [============================>.] - ETA: 0s - loss: 0.2586 - accuracy: 0.8912
Epoch 4: val_a

## 6. 모델 평가 및 예측

### 📊 성능 평가
저장된 최적 모델을 로드하여 테스트 데이터로 성능을 평가합니다.

### 🔮 실시간 예측
새로운 영화 리뷰에 대해 긍정/부정을 예측하는 파이프라인을 구성합니다.


In [15]:
# 최적 모델 로드 및 평가
print("📂 최적 모델 로드 중...")
best_model = models.load_model("binary_2gram_tfidf.keras")
print("✅ 모델 로드 완료!")

print("\n🧪 테스트 데이터셋 평가:")
print("-" * 40)

test_results = best_model.evaluate(binary_1gram_test_ds, verbose=1)
test_loss, test_accuracy = test_results

print(f"\n📊 최종 성능:")
print(f"   - 테스트 손실: {test_loss:.4f}")
print(f"   - 테스트 정확도: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")


📂 최적 모델 로드 중...
✅ 모델 로드 완료!

🧪 테스트 데이터셋 평가:
----------------------------------------
782/782 [==============================] - 38s 47ms/step - loss: 0.2996 - accuracy: 0.8848

📊 최종 성능:
   - 테스트 손실: 0.2996
   - 테스트 정확도: 0.8848 (88.48%)


In [16]:
# 실시간 예측 파이프라인 구성
print("🔮 실시간 예측 파이프라인 구성 중...")

# 문자열 입력 -> TF-IDF 벡터화 -> 예측 파이프라인
inputs = keras.Input(shape=(1,), dtype="string", name="text_input")
processed_inputs = text_vectorization(inputs)
outputs = best_model(processed_inputs)
inference_model = keras.Model(inputs, outputs, name="inference_pipeline")

print("✅ 예측 파이프라인 완성!")

# 테스트 리뷰들
test_reviews = [
    "That was an excellent movie, I loved it!",
    "This movie was terrible and boring.",
    "Amazing cinematography and great acting.",
    "Worst movie I've ever seen. Complete waste of time.",
    "Pretty good movie with some nice moments.",
    "Absolutely horrible! I want my money back."
]

print("\n🎭 영화 리뷰 감정 분석 결과:")
print("=" * 60)

for i, review in enumerate(test_reviews, 1):
    # 예측 수행
    markdown_text_data = tf.convert_to_tensor([[review]])
    prediction = inference_model(markdown_text_data)
    probability = prediction[0][0].numpy()
    
    # 결과 해석
    sentiment = "😊 긍정" if probability > 0.5 else "😞 부정"
    confidence = probability if probability > 0.5 else (1 - probability)
    
    print(f"\n리뷰 {i}: {review}")
    print(f"   → 예측: {sentiment}")
    print(f"   → 신뢰도: {confidence:.1%}")
    print(f"   → 긍정 확률: {probability:.1%}")


🔮 실시간 예측 파이프라인 구성 중...
✅ 예측 파이프라인 완성!

🎭 영화 리뷰 감정 분석 결과:

리뷰 1: That was an excellent movie, I loved it!
   → 예측: 😊 긍정
   → 신뢰도: 98.1%
   → 긍정 확률: 98.1%

리뷰 2: This movie was terrible and boring.
   → 예측: 😞 부정
   → 신뢰도: 86.7%
   → 긍정 확률: 13.3%

리뷰 3: Amazing cinematography and great acting.
   → 예측: 😊 긍정
   → 신뢰도: 91.6%
   → 긍정 확률: 91.6%

리뷰 4: Worst movie I've ever seen. Complete waste of time.
   → 예측: 😞 부정
   → 신뢰도: 96.3%
   → 긍정 확률: 3.7%

리뷰 5: Pretty good movie with some nice moments.
   → 예측: 😊 긍정
   → 신뢰도: 78.7%
   → 긍정 확률: 78.7%

리뷰 6: Absolutely horrible! I want my money back.
   → 예측: 😞 부정
   → 신뢰도: 80.0%
   → 긍정 확률: 20.0%


## 🎉 프로젝트 완료!

### 📈 성과 요약
- **데이터**: IMDb 영화 리뷰 50,000개 처리
- **방법**: TF-IDF + 2-gram 벡터화
- **모델**: 간단한 Dense Neural Network
- **결과**: 테스트 정확도 약 85-90% 달성 (일반적)

### 🔍 TF-IDF의 장점
- ✅ **빠른 훈련**: 사전 계산된 특성으로 빠른 학습
- ✅ **해석 가능**: 어떤 단어가 중요한지 확인 가능  
- ✅ **메모리 효율**: 희소 행렬로 효율적 저장
- ✅ **강건성**: N-gram으로 문맥 정보 캡처

### 🚀 개선 방향
- **워드 임베딩**: Word2Vec, GloVe 등 사용
- **딥러닝**: RNN, LSTM, Transformer 적용
- **앙상블**: 여러 모델 조합으로 성능 향상
- **하이퍼파라미터**: 그리드 서치로 최적화

---
> 💡 **다음 단계**: 임베딩 기반 모델(`자연어_임베딩1.py`, `자연어_임베딩2.py`)과 성능 비교해보세요!
